<a href="https://colab.research.google.com/github/brianmatute011/GColab/blob/main/TextClasificationBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl ipecho.net/plain

34.138.161.116

In [ ]:
!curl ipecho.net/plain

34.86.192.163

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=0a46f5fb6438e4f64d385187a4699e796ab9f52359038e5a23c8aad4c1bb0f8c
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.12.0
Eager mode:  True
Hub version:  0.13.0
GPU is NOT AVAILABLE


In [ ]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJO8T92/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJO8T92/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJO8T92/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
print(train_examples_batch[0])


tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)


In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
story = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=12,
                    validation_data=validation_data.batch(512),
                    verbose=1)


Epoch 1/12
30/30 [==============================] - 3s 64ms/step - loss: 1.0940 - accuracy: 0.4603 - val_loss: 0.8159 - val_accuracy: 0.4942
Epoch 2/12
30/30 [==============================] - 2s 49ms/step - loss: 0.7515 - accuracy: 0.5245 - val_loss: 0.6970 - val_accuracy: 0.5611
Epoch 3/12
30/30 [==============================] - 3s 86ms/step - loss: 0.6666 - accuracy: 0.5952 - val_loss: 0.6408 - val_accuracy: 0.6181
Epoch 4/12
30/30 [==============================] - 4s 104ms/step - loss: 0.6186 - accuracy: 0.6424 - val_loss: 0.6039 - val_accuracy: 0.6518
Epoch 5/12
30/30 [==============================] - 3s 96ms/step - loss: 0.5811 - accuracy: 0.6741 - val_loss: 0.5722 - val_accuracy: 0.6772
Epoch 6/12
30/30 [==============================] - 2s 64ms/step - loss: 0.5470 - accuracy: 0.7047 - val_loss: 0.5427 - val_accuracy: 0.7100
Epoch 7/12
30/30 [==============================] - 2s 50ms/step - loss: 0.5138 - accuracy: 0.7328 - val_loss: 0.5163 - val_accuracy: 0.7489
Epoch 8/12
3

results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.4083 - accuracy: 0.8029 - 2s/epoch - 34ms/step
loss: 0.408
accuracy: 0.803


In [ ]:
from sklearn.metrics import classification_report

# Obtener las predicciones del modelo en el conjunto de prueba
predictions = model.predict(test_data.batch(512))
predictions = tf.squeeze(tf.round(predictions))

# Obtener las etiquetas reales del conjunto de prueba
labels = np.concatenate([y for x, y in test_data.batch(512)], axis=0)

# Calcular el recall y el F1-score
#report = classification_report(labels, predictions)
#print(labels)

49/49 [==============================] - 1s 28ms/step
[1 1 0 ... 0 1 1]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Obtener un iterador para el conjunto de datos de prueba
test_iterator = iter(test_data)

# Obtener una muestra del conjunto de datos de prueba
num_samples = 20  # Cambia el número 10 por la cantidad de muestras que deseas obtener
test_samples = [next(test_iterator) for _ in range(num_samples)]

# Obtener las predicciones en la muestra de datos de prueba
test_predictions = model.predict(tf.stack([sample[0] for sample in test_samples]))

# Convertir las predicciones en etiquetas binarias
binary_predictions = [1 if pred >= 0.5 else 0 for pred in test_predictions]

# Obtener las etiquetas verdaderas de la muestra de datos de prueba
test_labels = [sample[1].numpy() for sample in test_samples]


1/1 [==============================] - 0s 64ms/step


In [ ]:
for i in range(len(test_labels)):
    if binary_predictions[i] == test_labels[i]:
        print("text:", test_samples[i][0].numpy())
        print("prediction:", binary_predictions[i])
        print("label:", test_labels[i])
        print("----------")

text: b"A blackly comic tale of a down-trodden priest, Nazarin showcases the economy that Luis Bunuel was able to achieve in being able to tell a deeply humanist fable with a minimum of fuss. As an output from his Mexican era of film making, it was an invaluable talent to possess, with little money and extremely tight schedules. Nazarin, however, surpasses many of Bunuel's previous Mexican films in terms of the acting (Francisco Rabal is excellent), narrative and theme.<br /><br />The theme, interestingly, is something that was explored again in Viridiana, made three years later in Spain. It concerns the individual's struggle for humanity and altruism amongst a society that rejects any notion of virtue. Father Nazarin, however, is portrayed more sympathetically than Sister Viridiana. Whereas the latter seems to choose charity because she wishes to atone for her (perceived) sins, Nazarin's whole existence and reason for being seems to be to help others, whether they (or we) like it or n